# Capstone Project: Detection + Tracking Pipeline

Guided project: dataset prep -> fine-tune detector -> run on video -> track objects -> evaluate.

## Deliverables
- `inference.py` sample
- `results/` annotated video
- explanation and next steps
